In [1]:


import mobster as mb

import numpy as np



Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
data = mb.load_example_data("./")

#data = data["1:1"]

In [3]:
data

{'1:0': tensor([0.5147, 0.4165, 0.4011, 0.4923, 0.5095, 0.5111, 0.4555, 0.4485, 0.0578,
         0.5306, 0.9053, 0.4660, 0.2190, 0.1007, 0.8990, 0.8770, 0.8634, 0.4526,
         0.4736, 0.5118, 0.0936, 0.0962, 0.1086, 0.8990, 0.4404, 0.5270, 0.5186,
         0.4384, 0.0485, 0.8611, 0.8831, 0.0920, 0.8561, 0.8561, 0.8561, 0.9734,
         0.9094, 0.4876, 0.8821, 0.1850, 0.9541, 0.0890, 0.9203, 0.9296, 0.0516,
         0.9290, 0.9316, 0.2744, 0.2343, 0.3844, 0.8697, 0.9190, 0.8879, 0.1007,
         0.8580, 0.8937, 0.8903, 0.9099, 0.8979, 0.3963, 0.9768, 0.8461, 0.8527,
         0.9060, 0.8657, 0.8615, 0.9019, 0.9231, 0.9506, 0.1874, 0.9332, 0.9514,
         0.9101, 0.8691, 0.9145, 0.9300, 0.9649, 0.0815, 0.0615, 0.9469, 0.8814,
         0.9384, 0.9069, 0.8468, 0.9240, 0.9402, 0.8806, 0.8990, 0.8462, 0.8820,
         0.9471, 0.9305, 0.9081, 0.0551, 0.0823, 0.3850, 0.9022, 0.9237, 0.9237,
         0.1101, 0.8979, 0.9108, 0.9286, 0.0769, 0.9390, 0.8740, 0.9174, 0.9345,
         0.9145, 0.85

In [4]:
fitted = mb.fit_mobster(data,1, 1,purity = 1,  truncated_pareto=False, max_it = 300, lr = 0.05,  e = 0.0001, lrd_gamma = 0.1,   
        number_of_trials_k= 100, prior_lims_k=[10,10000], prior_lims_clonal=[10.,10000.], number_of_trials_clonal_mean= 200.)

Running MOBSTER on 3 karyotypes with 1 subclones.
Fitting a model with tail

Computing cluster assignements.
Computing information criteria.
Done!


ELBO: -5790.185251799  :  11%|█         | 32/300 [00:01<00:08, 31.47it/s]


In [5]:

mb.plot_results(data, fitted, output= "cane.png", drivers=None)

In [6]:
fitted["model_parameters"]["2:1"]["cluster_assignments"].tolist()



['Tail',
 'Tail',
 'Tail',
 'Tail',
 'Tail',
 'Tail',
 'Tail',
 'C1',
 'C1',
 'S1',
 'C1',
 'Tail',
 'S1',
 'Tail',
 'Tail',
 'C1',
 'S1',
 'Tail',
 'Tail',
 'Tail',
 'Tail',
 'C1',
 'C1',
 'S1',
 'Tail',
 'C1',
 'C1',
 'Tail',
 'C1',
 'S1',
 'C1',
 'C1',
 'S1',
 'S1',
 'C1',
 'Tail',
 'C1',
 'C1',
 'C1',
 'S1',
 'C1',
 'C1',
 'S1',
 'Tail',
 'C1',
 'Tail',
 'C1',
 'C1',
 'C1',
 'C1',
 'C1',
 'C1',
 'C1',
 'Tail',
 'S1',
 'C1',
 'C1',
 'C1',
 'C1',
 'C1',
 'S1',
 'C1',
 'C1',
 'C1',
 'C1',
 'S1',
 'C1',
 'C1',
 'S1',
 'S1',
 'Tail',
 'S1',
 'S1',
 'Tail',
 'C2',
 'Tail',
 'C2',
 'C1',
 'C2',
 'C1',
 'S1',
 'S1',
 'C1',
 'C1',
 'C1',
 'C2',
 'C1',
 'C2',
 'C2',
 'C2',
 'S1',
 'C2',
 'C2',
 'Tail',
 'C2',
 'C1',
 'C2',
 'S1',
 'C2',
 'C1',
 'S1',
 'Tail',
 'C1',
 'S1',
 'C2',
 'S1',
 'C2',
 'S1',
 'C1',
 'C1',
 'C1',
 'C1',
 'S1',
 'C2',
 'C2',
 'C1',
 'Tail',
 'C1',
 'S1',
 'S1',
 'C1',
 'C1',
 'Tail',
 'C1',
 'S1',
 'C2',
 'C2',
 'C1',
 'C1',
 'C1',
 'C1',
 'S1',
 'Tail',
 'C1',
 'C2',

1